# Validates the SASKTRAN2 Voigt line broadening against the HITRAN API

In [4]:
import sasktran2 as sk
import numpy as np
import hapi
import matplotlib.pyplot as plt

pressures = np.array([101325, 10000, 10])
temperatures = np.array([280, 270, 260])

In [9]:
molecule = "H2O"
wavenumber =  np.arange(7000, 8000, 0.01)

sk2_xs = sk.optical.HITRANAbsorber(molecule).cross_sections(1e7 / wavenumber, np.ones_like(pressures), temperature_k=temperatures, pressure_pa=pressures, num_threads=1)

sk2_xs.shape

(100000, 3)

In [12]:
# Do the HITRAN calculation
line_db = sk.database.HITRANLineDatabase()
line_db.initialize_hapi(molecule)

hapi.select(
    molecule,
    DestinationTableName="spectral_window",
    Conditions=(
        "between",
        "nu",
        wavenumber[0] - 10,
        wavenumber[-1] + 10,
    ),
)

xs_hapi = np.zeros_like(sk2_xs)

for i, (P, T) in enumerate(zip(pressures, temperatures)):
    _, xs = hapi.absorptionCoefficient_Voigt(
        SourceTables="spectral_window",
        Environment={"T": T, "p": P / 101325.0},
        WavenumberGrid=wavenumber.tolist(),
        WavenumberWing=10,
    )
    xs_hapi[:, i] = xs / 1e4


                     Lines parsed: 486258
{'air': 1.0}
8.406902 seconds elapsed for abscoef; nlines = 30301
{'air': 1.0}
8.365512 seconds elapsed for abscoef; nlines = 30301
{'air': 1.0}
8.331525 seconds elapsed for abscoef; nlines = 30301


In [20]:
for i in range(len(pressures)):
    print("Mean Error: ", np.mean(np.abs((sk2_xs[:, i] - xs_hapi[:, i]) / xs_hapi[:, i] * 100)), "%")
    print("Std Error: ", np.std(np.abs((sk2_xs[:, i] - xs_hapi[:, i]) / xs_hapi[:, i] * 100)), "%")

Mean Error:  0.0006037367899750745 %
Std Error:  0.0006198262369700701 %
Mean Error:  0.000806210998993587 %
Std Error:  0.0007226351425375043 %
Mean Error:  0.0011293099067711146 %
Std Error:  0.0008208076129146743 %
